In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [ ]:
import pandas as pd
import numpy as np

In [ ]:
df_c = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/AFD/Bao_28_items_1990_2023_v3.csv')
print(df_c.head())
df_c_lenght= len(df_c)
print(df_c_lenght)
print(df_c.dtypes) # Check the data types

In [ ]:
df_t = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/AFD/fraud_constr_dist.csv')
print(df_t.head())
df_t_lenght= len(df_t)
print(df_t_lenght)

In [ ]:
# Padding cik in text data
df_t['cik'] = df_t['cik'].astype(str).str.zfill(10)
print(df_t)

In [ ]:
# Create a new column by combining cik and year
df_t['cik_year'] = df_t['cik'] + '_' + df_t['year'].astype(str)
print(df_t)

In [ ]:
# Count NaN values in the 'cik' column in compustat data
nan_count_before = df_c['cik'].isna().sum()
df_c = df_c.dropna(subset=['cik'])  # Drops row NaN cik
nan_count_after = df_c['cik'].isna().sum()

# Convert cik to integers to remove decimal points
df_c['cik'] = df_c['cik'].astype(int)

# Padding cik
df_c['cik'] = df_c['cik'].astype(str).str.zfill(10)

# Convert fyear to string
df_c['fyear'] = df_c['fyear'].astype(str)

# Create a new column by combining cik and fyear
df_c['cik_fyear'] = df_c['cik'] + '_' + df_c['fyear']

# Print the number of NaN values
print(f'Number of NaN values in "cik" before preprocessing: {nan_count_before}')
print(f'Number of NaN values in "cik" after preprocessing: {nan_count_after}')

print(df_c)

In [ ]:
# Merge dataframes on 'cik_fyear' and 'cik_year'
merged_df = pd.merge(df_c, df_t, left_on='cik_fyear', right_on='cik_year')

print(merged_df.keys())

Index(['gvkey', 'datadate', 'fyear', 'indfmt', 'consol', 'popsrc', 'datafmt',
       'tic', 'cusip', 'conm',
       ...
       'responsible', 'motive', 'conviction', 'applicant', 'guilty',
       'reprehensible', 'threat', 'restricting', 'selfishness', 'cik_year'],
      dtype='object', length=510)


In [ ]:
# Count the length of the merged dataframe
merged_df_length = len(merged_df)
print(f'Length merged dataframe: {merged_df_length}')

Length merged dataframe: 1355


In [ ]:
# Create a new column combining 'cik' and 'year'
df_t['ticker_year'] = df_t['ticker'] + '_' + df_t['year'].astype(str)

# Convert 'fyear' to string
df_c['fyear'] = df_c['fyear'].astype(str)

# Create a new column combining 'cik' and 'fyear'
df_c['tic_fyear'] = df_c['tic'] + '_' + df_c['fyear']

# Merge dfs on ticker
merged_df_ticker = pd.merge(df_c, df_t, left_on='tic_fyear', right_on='ticker_year')
print(merged_df_ticker.keys())

Index(['gvkey', 'datadate', 'fyear', 'indfmt', 'consol', 'popsrc', 'datafmt',
       'tic', 'cusip', 'conm',
       ...
       'motive', 'conviction', 'applicant', 'guilty', 'reprehensible',
       'threat', 'restricting', 'selfishness', 'cik_year', 'ticker_year'],
      dtype='object', length=512)


In [ ]:
# Count the length of the merged dfs
merged_df__ticker_length = len(merged_df_ticker)

# Print the length of the merged dfs
print(f'Length merged dataframe: {merged_df__ticker_length}')

Length merged dataframe: 31479


In [ ]:
# Identify ticker years in df_t but not in merged_df_ticker
missing_ticker_years = df_t[~df_t['ticker_year'].isin(merged_df_ticker['ticker_year'])]

# Export
missing_ticker_years.to_csv('/content/drive/MyDrive/Colab Notebooks/AFD/missing_ticker_years.csv', index=False)

In [ ]:
# Read the cik ticker sec mapping
df_m = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/AFD/company_tickers.csv')
print(df_m)

       cik_str ticker                           title
0       320193   AAPL                      Apple Inc.
1       789019   MSFT                  MICROSOFT CORP
2      1652044  GOOGL                   Alphabet Inc.
3      1018724   AMZN                  AMAZON COM INC
4      1045810   NVDA                     NVIDIA CORP
...        ...    ...                             ...
10904  1823882  MIMWW  Airspan Networks Holdings Inc.
10905   874710  AHPIQ  ALLIED HEALTHCARE PRODUCTS INC
10906  1843862  EOCWF    Elliott Opportunity II Corp.
10907  1062750  SAAYY                SAIPEM S P A /FI
10908  1733861   RTEZ                    REST EZ Inc.

[10909 rows x 3 columns]


In [ ]:
# create a new df with the new cik from the mapping
merged_df3 = pd.merge(df_t, df_m[['ticker', 'cik_str']], on='ticker', how='left')


In [ ]:
print(merged_df3)

       Unnamed: 0     id ticker         cik  year  accession_number  \
0               0      1   AAPL  0000320193  2017                70   
1               1      2   AAPL  0000320193  2018               145   
2               2      3   AAPL  0000320193  2019               119   
3               3      4   AAPL  0000320193  2020                96   
4               4      5   AAPL  0000320193  2021               105   
...           ...    ...    ...         ...   ...               ...   
48828       48828  48829  TNONW  0001575872  2023               346   
48829       48829  48830  MIMWW  0001829126  2021              2055   
48830       48830  48831  MIMWW  0001829126  2022              7958   
48831       48831  48832   RTEZ  0001185185  2022               827   
48832       48832  48833   RTEZ  0001185185  2023               944   

       mda_section                              target_word_frequency  \
0                0  {"recklessness": 0, "guarantor": 0, "hazard": ...   
1

In [ ]:
merged_df3.to_csv('/content/drive/MyDrive/Colab Notebooks/AFD/merged_new_cik.csv')

In [ ]:
# Read the new text file with the new cik and the compustat data
df_t2 = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/AFD/merged_new_cik.csv')
df_c = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/AFD/Bao_28_items_1990_2023_v3.csv')
print(df_t2.keys())
print(df_c.keys())

Index(['Unnamed: 0.1', 'Unnamed: 0', 'id', 'ticker', 'cik', 'year',
       'accession_number', 'mda_section', 'target_word_frequency',
       'created_at',
       ...
       'conviction', 'applicant', 'guilty', 'reprehensible', 'threat',
       'restricting', 'selfishness', 'cik_year', 'ticker_year', 'cik_str'],
      dtype='object', length=467)
Index(['gvkey', 'datadate', 'fyear', 'indfmt', 'consol', 'popsrc', 'datafmt',
       'tic', 'cusip', 'conm', 'curcd', 'fyr', 'act', 'ap', 'at', 'ceq', 'che',
       'cogs', 'csho', 'dlc', 'dltis', 'dltt', 'dp', 'ib', 'invt', 'ivao',
       'ivst', 'lct', 'lt', 'ni', 'ppegt', 'ppent', 'pstk', 're', 'rect',
       'sale', 'sstk', 'txp', 'txt', 'xint', 'cik', 'costat', 'prcc_f',
       'conml', 'sic'],
      dtype='object')


In [ ]:
# Drop rows with NaN 'cik' and convert to int
df_c = df_c.dropna(subset=['cik'])
df_c['cik'] = pd.to_numeric(df_c['cik'], errors='coerce')
df_c = df_c.dropna(subset=['cik'])
df_c['cik'] = df_c['cik'].astype(int)

<ipython-input-18-03da7d4bf7ae>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_c['cik'] = pd.to_numeric(df_c['cik'], errors='coerce')


In [ ]:
# Padding cik
df_t2['cik_str'] = df_t2['cik_str'].astype(str).str.zfill(10)
df_c['cik'] = df_c['cik'].astype(str).str.zfill(10)

In [ ]:
# Convert year to string
df_t2['year'] = df_t2['year'].astype(str)
df_c['fyear'] = df_c['fyear'].astype(str)

In [ ]:
# Create new column cik year in both df
df_t2['cik_year2'] = df_t2['cik_str'] + '_' + df_t2['year']
df_c['cik_year2'] = df_c['cik'] + '_' + df_c['fyear']

print(df_t2)
print(df_c)


       Unnamed: 0.1  Unnamed: 0     id ticker      cik  year  \
0                 0           0      1   AAPL   320193  2017   
1                 1           1      2   AAPL   320193  2018   
2                 2           2      3   AAPL   320193  2019   
3                 3           3      4   AAPL   320193  2020   
4                 4           4      5   AAPL   320193  2021   
...             ...         ...    ...    ...      ...   ...   
48828         48828       48828  48829  TNONW  1575872  2023   
48829         48829       48829  48830  MIMWW  1829126  2021   
48830         48830       48830  48831  MIMWW  1829126  2022   
48831         48831       48831  48832   RTEZ  1185185  2022   
48832         48832       48832  48833   RTEZ  1185185  2023   

       accession_number  mda_section  \
0                    70            0   
1                   145            0   
2                   119            0   
3                    96            0   
4                   105        

In [ ]:
# Create both files by cik
merged_df_cik2 = pd.merge(df_c, df_t2, left_on='cik_year2', right_on='cik_year2')
print(merged_df_cik2.keys())

merged_df_cik2_length = len(merged_df_cik2)
print(f'Length merged dataframe: {merged_df_cik2_length}')

Index(['gvkey', 'datadate', 'fyear', 'indfmt', 'consol', 'popsrc', 'datafmt',
       'tic', 'cusip', 'conm',
       ...
       'conviction', 'applicant', 'guilty', 'reprehensible', 'threat',
       'restricting', 'selfishness', 'cik_year', 'ticker_year', 'cik_str'],
      dtype='object', length=513)
Length merged dataframe: 37977


In [ ]:
# Identify cik2 years in df_t but not in merged_df_cik2 and export
missing_cik_years2 = df_t2[~df_t2['cik_year2'].isin(merged_df_cik2['cik_year2'])]

missing_cik_years2.to_csv('/content/drive/MyDrive/Colab Notebooks/AFD/missing_cik_years2.csv', index=False)

In [ ]:
# Compare the 37977 cik matches by ticker now
# Create a new column combining 'cik' and 'fyear'
df_c = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/AFD/Bao_28_items_1990_2023_v3.csv')
df_c['fyear'] = df_c['fyear'].astype(str)
df_c['tic_fyear'] = df_c['tic'] + '_' + df_c['fyear']
ticker_merge = pd.merge(missing_cik_years2, df_c, left_on='ticker_year', right_on='tic_fyear')
print(ticker_merge)

ticker_merge_length = len(ticker_merge)
print(f'Length merged dataframe: {ticker_merge_length}')

     Unnamed: 0.1  Unnamed: 0     id ticker    cik_x  year  accession_number  \
0            1665        1665   1666    GLD   950123  2008             16306   
1            1666        1666   1667    GLD   950123  2009             66100   
2            1667        1667   1668    GLD   950123  2010            107692   
3            1668        1668   1669    GLD   950123  2011            100014   
4            1669        1669   1670    GLD   950136  2005              8184   
..            ...         ...    ...    ...      ...   ...               ...   
683         47172       47172  47173    SCO  1193125  2018             67452   
684         47173       47173  47174    SCO  1193125  2019             60564   
685         47174       47174  47175    SCO  1193125  2020             56815   
686         47175       47175  47176    SCO  1193125  2021             49128   
687         47176       47176  47177    SCO  1193125  2022             54559   

     mda_section                       

In [ ]:
cik_ticker_match_df = merged_df_cik2.append(ticker_merge, ignore_index=True)
cik_ticker_match_df.to_csv('/content/drive/MyDrive/Colab Notebooks/AFD/cik_ticker_match_df.csv')

<ipython-input-25-21657c710bca>:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  cik_ticker_match_df = merged_df_cik2.append(ticker_merge, ignore_index=True)


In [ ]:
cik_ticker_match_df_length = len(cik_ticker_match_df)
print(f'Length merged dataframe: {cik_ticker_match_df_length}')

Length merged dataframe: 38665


In [ ]:
count_2023 = len(missing_cik_years2[missing_cik_years2['year'] == '2023'])
print(f'Number of rows with "year" equal to 2023: {count_2023}')

Number of rows with "year" equal to 2023: 3846
